In [2]:
import os
import utilities
from maths_objects import (
    Formulas, SiteHierarchies,MarkdownFile, DataSource, FrontMatter)
import pandas as pd


data_source = DataSource()
site_hierarchy = SiteHierarchies(data_source.site_hierarchies())
formulas_by_year = Formulas(data_source.formulas_by_year())

In [14]:
by_year_hierarchies = site_hierarchy.formulas_by_year_ex_summaries_file_paths()

for path in by_year_hierarchies:
    print (path.state)



NSW
NSW
NSW
NSW
NSW
NSW
NSW
NSW
NSW
NSW


In [44]:
from collections import namedtuple

def formulas_by_year_ex_summaries_file_paths(self):
    """Returns formulas by year file paths as a list of strings.
    Exlucdes file paths designated as summries"""
    file_paths = self._hierarchy_data.copy()
    file_paths = file_paths[
        (file_paths['Level_1'] == 'Formulas') &
        (file_paths['Level_2'] == 'By year') &
        (file_paths['Level_4'] != 'Summaries')
    ]
    file_paths = file_paths.apply(
        lambda x: os.path.sep.join(list(x.dropna())), axis='columns')
    return file_paths

def _to_named_tuple(hierarchy):

    return_value = namedtuple('Hiearchy', ['state', 'subject', 
                                                'category', 'path'])
    return_value.state = hierarchy['Level_0']
    return_value.subject = hierarchy['Level_3']
    return_value.category = hierarchy['Level_4']
    return_value.path = os.path.sep.join(list(hierarchy.dropna()))
    return return_value

path_as_series = pd.Series('NSW/Formulas/By year/Year 9/Surds'.split(os.path.sep))
path_as_series.index = ['Level_0','Level_1','Level_2','Level_3','Level_4']

x = formulas_by_year_ex_summaries_file_paths(site_hierarchy)

y = _to_named_tuple(path_as_series)
y.state

'NSW'

In [24]:
site_hierarchy.formulas_by_year_ex_summaries_file_paths()

10                    NSW/Formulas/By year/Year 9/Surds
11                  NSW/Formulas/By year/Year 9/Indices
12          NSW/Formulas/By year/Year 11 Adv/Logarithms
13     NSW/Formulas/By year/Year 11 Adv/Differentiation
14     NSW/Formulas/By year/Year 12 Adv/Differentiation
15         NSW/Formulas/By year/Year 12 Adv/Integration
16          NSW/Formulas/By year/Year 12 Adv/Statistics
17    NSW/Formulas/By year/Year 12 Adv/Financial mat...
19    NSW/Formulas/By year/Year 12 Ext 1/Differentia...
20       NSW/Formulas/By year/Year 12 Ext 1/Integration
dtype: object

In [25]:
formulas_by_year.to_dataframe()

,State,Subject,Syllabus topic,Syllabus subtopic code,Syllabus subtopic,Category,Subcategory_1,Subcategory_2,Description,Group,Formula,On formula sheet,Proof required,Comment
0,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Multiplying terms with same base,NaN,$a^m \times a^n = a^{m+n}$,False,False,NaN
1,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Dividing terms with same base,NaN,$a^m \div a^n = \dfrac{a^m}{a^n} = a^{m-n}$,False,False,NaN
2,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Power of a power,NaN,$(a^m)^n = a^{m \times n}$,False,False,NaN
3,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Powers of products,NaN,$(ab)^n = a^nb^n$,False,False,NaN
4,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Powers of quotients,NaN,$\left(\dfrac{a}{b}\right)^n = \dfrac{a^n}{b^n}$,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,Financial mathematics,Arithmetic sequence,Sum of first n terms,Sum of the first n terms of an arithmetic sequ...,NaN,$S_n = \dfrac{n}{2}[2a + (n-1)d] = \dfrac{n}{2...,True,True,NaN
68,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,Financial mathematics,Geometric sequence,Recursive definition,Recursive definition of a geometric sequence,NaN,$T_n = rT_{n-1}$,False,False,NaN
69,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,Financial mathematics,Geometric sequence,n-th term,n-th term of a geometric sequence,NaN,$T_n = ar^{n-1}$,True,True,NaN
70,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,Financial mathematics,Geometric sequence,Sum of first n terms,Sum of the first n terms of a geometric sequence,NaN,$S_n = \dfrac{a(1-r^n)}{1-r} = \dfrac{a(r^n-1)...,True,True,NaN


In [11]:
def set_styled_table_widths(styled_table, widths):
    """Sets pandas dataframe stlyle column withs where widths is represents a
    dict of column names and widths in pixels as integers.  Ignores column
    names that do not exist.  Purpously does not generate an error to allow
    the function to be called for a broader set of column names without
    requiring a check in the calling sub"""
    return_table = styled_table
    for column_name, width in widths.items():
        if column_name in styled_table.columns:
            return_table = return_table.set_properties(
                subset=[column_name], **{'width': str(width) + 'px'})
    return return_table



def is_on_formula_sheet_formatting(formula, formula_sheet_items):
    """Returns formatting for pandas styler object based on whether formulas
    is contained in formula_sheet_items (pandas series)"""
    if formula in formula_sheet_items.values:
        return 'background-color:rgba(255,194,10, 0.2);'
    else:
        return None


def is_on_formula_proof_required_formatting(formula,
                                            formula_proof_required_items):
    """Returns formatting for pandas styler object based on whether formulas
    is contained in formula_sheet_items (pandas series)
    """
    if formula in formula_proof_required_items.values:
        return 'background-color:rgba(0,150,200, 0.2);'
    else:
        return None


def df_to_formula_styled_table(input_df, col_widths=None,
                               cols_to_highlight=None,
                               formula_sheet_items=pd.Series(dtype="string"),
                               formula_proof_required_items=(
                                   pd.Series(dtype="string")),
                               display_col_headers=True,
                               display_row_headers=True):
    """Converts pandas dataframe to a styler and applies various formatting
    Note that index of df needs to be unique"""

    if col_widths is None:
        col_widths = {}
    if cols_to_highlight is None:
        cols_to_highlight = []

    styled_table = input_df.fillna('').style
    styled_table = set_styled_table_widths(styled_table, col_widths)

    if not display_col_headers:
        styled_table = styled_table.hide(axis='columns')
    if not display_row_headers:
        styled_table = styled_table.hide(axis='index')

    # Calculate intersect with df columns to avoid error if cols not in df
    cols_to_highlight = list(set(cols_to_highlight)
                             & set(list(input_df.columns)))
    if not formula_sheet_items.empty:
        for col in cols_to_highlight:
            styled_table = styled_table.applymap(
                is_on_formula_sheet_formatting,
                subset=[col],
                formula_sheet_items=formula_sheet_items)
    if not formula_proof_required_items.empty:
        for col in cols_to_highlight:
            styled_table = styled_table.applymap(
                is_on_formula_proof_required_formatting,
                subset=[col],
                formula_proof_required_items=formula_proof_required_items)

    styled_table = styled_table.set_table_styles([
        {'selector': 'th.col_heading', 'props':
         'text-align: left; font-size:1em;'},
        {'selector': 'td',
         'props': 'text-align: left; font-size:1em;padding: 1.5em;'}])

    # below allows newlines in the csv, outside of the latex dollar signs to
    # be reflected on display
    styled_table = styled_table.set_properties(**{'white-space': 'pre-wrap'})

    return styled_table

In [25]:

for formula_page in formulas_by_year.formula_tables():
    print (os.path.sep.join([
       formula_page.state(), formula_page.subject(), formula_page.category ()
    ]))



NSW/Year 9/Indices
NSW/Year 11 Adv/Logarithms
NSW/Year 11 Adv/Differentiation
NSW/Year 12 Adv/Differentiation
NSW/Year 12 Adv/Integration
NSW/Year 12 Ext 1/Differentiation
NSW/Year 12 Ext 1/Integration
NSW/Year 12 Adv/Statistics
NSW/Year 9/Surds
NSW/Year 12 Adv/Financial mathematics


In [24]:
data_source.formulas_by_year()

,State,Subject,Syllabus topic,Syllabus subtopic code,Syllabus subtopic,Category,Subcategory_1,Subcategory_2,Description,Group,Formula,On formula sheet,Proof required,Comment
0,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Multiplying terms with same base,NaN,$a^m \times a^n = a^{m+n}$,True,False,NaN
1,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Dividing terms with same base,NaN,$a^m \div a^n = \dfrac{a^m}{a^n} = a^{m-n}$,True,False,NaN
2,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Power of a power,NaN,$(a^m)^n = a^{m \times n}$,True,False,NaN
3,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Powers of products,NaN,$(ab)^n = a^nb^n$,True,False,NaN
4,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Powers of quotients,NaN,$\left(\dfrac{a}{b}\right)^n = \dfrac{a^n}{b^n}$,True,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,Financial mathematics,Arithmetic sequence,Sum of first n terms,Sum of the first n terms of an arithmetic sequ...,NaN,$S_n = \dfrac{n}{2}[2a + (n-1)d] = \dfrac{n}{2...,True,True,NaN
68,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,Financial mathematics,Geometric sequence,Recursive definition,Recursive definition of a geometric sequence,NaN,$T_n = rT_{n-1}$,True,False,NaN
69,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,Financial mathematics,Geometric sequence,n-th term,n-th term of a geometric sequence,NaN,$T_n = ar^{n-1}$,True,True,NaN
70,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,Financial mathematics,Geometric sequence,Sum of first n terms,Sum of the first n terms of a geometric sequence,NaN,$S_n = \dfrac{a(1-r^n)}{1-r} = \dfrac{a(r^n-1)...,True,True,NaN
